# DSFB Simulation Visualization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/infinityabundance/dsfb/blob/main/crates/dsfb/dsfb_simulation.ipynb)

This notebook visualizes the results of the DSFB (Drift-Slew Fusion Bootstrap) simulation.

## Instructions

To run this notebook:
1. Clone the repository locally
2. Run: `cargo run --release -p dsfb --example drift_impulse` to generate `output-dsfb/<timestamp>/sim-dsfb.csv`
3. In Colab, this notebook auto-detects the newest `output-dsfb/<timestamp>/sim-dsfb.csv`; you can also upload your own CSV
4. Run all cells

Legacy fallback paths (`sim.csv`, `out/sim.csv`) remain supported.


In [ ]:
# Install required packages if needed
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
except ImportError:
    !pip install pandas matplotlib numpy
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np

In [ ]:
# Locate simulation CSV (latest timestamped run, local file, downloaded sample, or uploaded file)
from pathlib import Path
import os

def find_latest_timestamped_csv(base_dir: Path):
    if not base_dir.exists():
        return None
    preferred = sorted(base_dir.glob('*/sim-dsfb.csv'))
    if preferred:
        return preferred[-1]
    legacy = sorted(base_dir.glob('*/sim.csv'))
    if legacy:
        return legacy[-1]
    return None

env_csv = os.environ.get('DSFB_SIM_CSV', '').strip()
csv_path = None

if env_csv and Path(env_csv).exists():
    csv_path = env_csv

if csv_path is None:
    latest = find_latest_timestamped_csv(Path('output-dsfb'))
    if latest is not None:
        csv_path = str(latest)

if csv_path is None:
    static_candidates = [
        Path('output-dsfb/sim-dsfb.csv'),
        Path('sim-dsfb.csv'),
        Path('crates/dsfb/sim-dsfb.csv'),
        Path('sim.csv'),
        Path('crates/dsfb/sim.csv'),
        Path('output-dsfb/sim.csv'),
        Path('out/sim.csv'),
    ]
    for candidate in static_candidates:
        if candidate.exists():
            csv_path = str(candidate)
            break

if csv_path is None:
    sample_urls = [
        'https://raw.githubusercontent.com/infinityabundance/dsfb/main/crates/dsfb/sim.csv',
        'https://raw.githubusercontent.com/infinityabundance/dsfb/main/notebooks/sim.csv',
    ]
    for sample_url in sample_urls:
        try:
            import urllib.request
            urllib.request.urlretrieve(sample_url, 'sim-dsfb.csv')
            csv_path = 'sim-dsfb.csv'
            print(f'Downloaded sample CSV from: {sample_url}')
            break
        except Exception:
            pass

if csv_path is None:
    try:
        from google.colab import files
        print('Upload sim-dsfb.csv (or output-dsfb/<timestamp>/sim-dsfb.csv) from your machine...')
        uploaded = files.upload()
        if not uploaded:
            raise FileNotFoundError('No file uploaded.')
        csv_path = next(iter(uploaded.keys()))
    except Exception as err:
        raise FileNotFoundError(
            'Could not find a DSFB simulation CSV. '
            'Expected sim-dsfb.csv or output-dsfb/<timestamp>/sim-dsfb.csv. '
            'Generate it with: cargo run --release -p dsfb --example drift_impulse'
        ) from err

print(f'Using CSV: {csv_path}')


In [ ]:
# Load simulation data
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} time steps")
print(f"Time range: {df['t'].min():.2f} to {df['t'].max():.2f}")
df.head()

In [ ]:
# Plot: True state vs estimates
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df['t'], df['phi_true'], 'k-', linewidth=2, label='True φ', alpha=0.8)
ax.plot(df['t'], df['phi_mean'], 'b--', linewidth=1.5, label='Mean Fusion', alpha=0.7)
ax.plot(df['t'], df['phi_freqonly'], 'g--', linewidth=1.5, label='Freq-Only', alpha=0.7)
ax.plot(df['t'], df['phi_dsfb'], 'r-', linewidth=1.5, label='DSFB', alpha=0.8)

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('φ (position)', fontsize=12)
ax.set_title('Position Estimates Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot: Error curves
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df['t'], df['err_mean'], 'b-', linewidth=1.5, label='Mean Fusion Error', alpha=0.7)
ax.plot(df['t'], df['err_freqonly'], 'g-', linewidth=1.5, label='Freq-Only Error', alpha=0.7)
ax.plot(df['t'], df['err_dsfb'], 'r-', linewidth=1.5, label='DSFB Error', alpha=0.8)

# Mark impulse region
impulse_start = 3.0  # Adjust based on actual simulation config
impulse_end = 4.0
ax.axvspan(impulse_start, impulse_end, alpha=0.2, color='orange', label='Impulse Period')

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Absolute Error', fontsize=12)
ax.set_title('Estimation Errors', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
plt.tight_layout()
plt.show()

In [ ]:
# Plot: Trust weight and EMA residual for channel 2
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Trust weight
ax1.plot(df['t'], df['w2'], 'purple', linewidth=2, label='Channel 2 Trust Weight')
ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Equal Weight')
ax1.axvspan(impulse_start, impulse_end, alpha=0.2, color='orange', label='Impulse Period')
ax1.set_ylabel('Weight w₂', fontsize=12)
ax1.set_title('Trust Weight Adaptation', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# EMA residual
ax2.plot(df['t'], df['s2'], 'orange', linewidth=2, label='Channel 2 EMA Residual')
ax2.axvspan(impulse_start, impulse_end, alpha=0.2, color='orange', label='Impulse Period')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('EMA Residual s₂', fontsize=12)
ax2.set_title('Residual Tracking', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compute and display metrics
def rms_error(errors):
    return np.sqrt(np.mean(errors**2))

rms_mean = rms_error(df['err_mean'])
rms_freqonly = rms_error(df['err_freqonly'])
rms_dsfb = rms_error(df['err_dsfb'])

# Find impulse indices
impulse_mask = (df['t'] >= impulse_start) & (df['t'] < impulse_end)
peak_mean = df.loc[impulse_mask, 'err_mean'].max()
peak_freqonly = df.loc[impulse_mask, 'err_freqonly'].max()
peak_dsfb = df.loc[impulse_mask, 'err_dsfb'].max()

# Create metrics table
metrics_df = pd.DataFrame({
    'Method': ['Mean Fusion', 'Freq-Only', 'DSFB'],
    'RMS Error': [rms_mean, rms_freqonly, rms_dsfb],
    'Peak Error (Impulse)': [peak_mean, peak_freqonly, peak_dsfb]
})

print("\n" + "="*60)
print("PERFORMANCE METRICS")
print("="*60)
print(metrics_df.to_string(index=False))
print("="*60)

# Calculate improvement
improvement_vs_mean = (rms_mean - rms_dsfb) / rms_mean * 100
improvement_vs_freqonly = (rms_freqonly - rms_dsfb) / rms_freqonly * 100

print(f"\nDSFB Improvement:")
print(f"  vs Mean Fusion: {improvement_vs_mean:.1f}% reduction in RMS error")
print(f"  vs Freq-Only:   {improvement_vs_freqonly:.1f}% reduction in RMS error")

In [ ]:
# Summary statistics
print("\nSummary Statistics:")
print(f"Minimum trust weight w₂: {df['w2'].min():.4f}")
print(f"Maximum trust weight w₂: {df['w2'].max():.4f}")
print(f"Mean trust weight w₂: {df['w2'].mean():.4f}")
print(f"\nMaximum EMA residual s₂: {df['s2'].max():.4f}")
print(f"Final EMA residual s₂: {df['s2'].iloc[-1]:.4f}")